## Tutorial: Second-Order CPA Attack
This tutorial demonstrates how to perform a second-order Correlation Power Analysis (SO-CPA) attack using the accelerated CPA implementation.

### Overview
In this tutorial, we will use the ASCAD dataset as the target for the SO-CPA attack. The workflow consists of the following steps:
1. **Download the ASCAD dataset.**
2. **Convert the dataset to ChipWhisperer Project format.**
3. **Setup a SO-CPA attack and execute it.**

## Preliminary knowledge

Various types of high-order CPA attacks have been proposed in the literature. Our implementation follows the product combining method, which is known to be the most efficient for second-order attacks. For more details, refer to the following paper:

- Prouff, Emmanuel, Matthieu Rivain, and Régis Bevan. "Statistical analysis of second order differential power analysis." *IEEE Transactions on Computers* 58.6 (2009): 799-811.

Assume that $N$ traces are available for the attack. When focusing on two sample points $t_1$ and $t_2$, we have two vectors of power traces $\mathbf{W}(t_1)$ and $\mathbf{W}(t_2)$, each of size $N$. 

The simplest way to combine these vectors is through element-wise multiplication:
$$
\mathbf{C}(t_1, t_2) = \mathbf{W}(t_1) \odot \mathbf{W}(t_2)
$$

For a more efficient combination, each element of the vector is subtracted by the mean of the vector:
$$
\mathbf{C}(t_1, t_2) = (\mathbf{W}(t_1) - \bar{\mathbf{W}}(t_1)) \odot (\mathbf{W}(t_2) - \bar{\mathbf{W}}(t_2))
$$

This is known as the normalized product combining method. Our SO-CPA library implements the normalized product combining approach to enhance the efficiency of second-order attacks.
However, if we do not limit the number of combined sample points, the number of produced vectors — and consequently, the number of correlation computations — grows quadratically with the number of sample points. For instance, if we have $T$ sample points, we will need to compute $T(T-1)/2$ correlations. To address this, a window size is introduced, as described in the following paper:

- Bottinelli, Paul, and Joppe W. Bos. "Computational aspects of correlation power analysis." *Journal of Cryptographic Engineering* 7 (2017): 167-181.

It is based on the observation that meaningful combined points are often close to each other. However, selecting a window size that is too small may result in the loss of significant correlations. Therefore, the window size should be chosen carefully, taking into account the target device and the specific attack scenario.

## Prerequisites
To convert the ASCAD dataset to ChipWhisperer project format, you need to get the following Python packages installed:
```bash
pip install numpy h5py
```

## Step 1: Download and Prepare the ASCAD Dataset

The first step is to download the ASCAD dataset, which contains power traces for side-channel analysis. You can obtain it from the [ASCAD repository](https://github.com/ANSSI-FR/ASCAD).

We will use the fixed key version of the ATMEGA power traces: `ASCAD/ATMEGA_AES_v1/ATM_AES_v1_fixed_key`. Follow the instructions in the repository to download the dataset. Ensure that the file `ATMega8515_raw_traces.h5` is located in the unzipped directory `ASCAD_data/ASCAD_databases`.

### Dataset Overview
- The raw dataset contains **60,000 traces**, each with **100,000 samples**.
- According to the dataset description, **700 samples** in the range `(45400, 46100)` are selected as the attack target. This range corresponds to the processing of the **2nd key byte** of AES encryption (zero-based index). Other key bytes cannot be recovered.
- Out of the 60,000 traces:
	- **50,000 traces** are used for profiling.
	- The remaining **10,000 traces** are used for the attack.
Therefore, we will use only the last 10,000 traces for the attack.

### Instructions
Run the following code block to select the `ATMega8515_raw_traces.h5` file. A file chooser will be displayed. Once you select a valid file, a checkmark will confirm the selection.


In [ ]:
from ipyfilechooser import FileChooser
import ipywidgets as widgets
import os
dataset_chooser = FileChooser()
dataset_status = widgets.Valid(
    value=False,
)
db_file = None
def check_dataset_selection():
    global db_file
    dataset_status.value = False
    if not dataset_chooser.selected:
        return
    selected_path = dataset_chooser.selected
    # check if the selected file is a valid dataset
    db_file = os.path.abspath(selected_path)
    if not os.path.exists(db_file):
        return
    dataset_status.value = True

dataset_chooser.register_callback(check_dataset_selection)
dataset_chooser.filter_pattern = "ATMega8515_raw_traces.h5"
display(widgets.HBox([widgets.Label("Dataset check"), dataset_status]))
display(dataset_chooser)

In [ ]:
import h5py
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import chipwhisperer as cw
from chipwhisperer.common.api import ProjectFormat as Project

if not dataset_status.value:
    print("Please select a valid dataset file.")
else:
    # Load the ASCAD Raw dataset
    database = h5py.File(db_file, 'r')

    point_range = slice(45400, 46100)
    trace_range = slice(50000, 60000)

    database_valid = False

    # check the structure of the dataset
    try:
        keys = database["metadata"]["key"]
        plaintext = database["metadata"]["plaintext"]
        ciphertext = database["metadata"]["ciphertext"]
        traces = database["traces"]

        if len(traces) != 60000:
            raise ValueError("The number of traces in the dataset is not 60000.")
        if len(traces[0]) != 100000:
            raise ValueError("The length of each trace in the dataset is not 100000.")
    except KeyError as e:
        print(f"KeyError: {e}. Please ensure the dataset is correctly formatted.")
        database.close()
    except ValueError as e:
        print(f"ValueError: {e}. Please check the dataset structure.")
        database.close()
    else:
        database_valid = True

    # convert the dataset to a ChipWhisperer project
    if database_valid:
        project = Project.Project()
        for i in tqdm(range(trace_range.start, trace_range.stop), desc="Processing traces"):
            wave = traces[i][point_range]
            project.traces.append(cw.Trace(wave, plaintext[i], ciphertext[i], keys[i]))

        # display the first 10 traces
        for i in range(10):
            plt.plot(project.traces[i].wave)
        plt.show()
    database.close()

### Optional step: Save the Project
If you save the converted project for future use, you can skip the conversion step next time.
Run the following code block to save the project. File chooser will be displayed.
You need to select the directory where you want to save the project and specify the project name.
After saving, `project_name.cwp` file and `project_name_data` directory will be created in the selected directory.

In [ ]:
from ipywidgets import Label
project_choose = FileChooser()

msg = Label()
def save_project():
    # check if the selected path is directory
    if not project_choose.selected:
        return
    if os.path.isdir(project_choose.selected):
        msg.value = "Please fill project name in output filename field."
        return
    project_path = Project.ensure_cwp_extension(project_choose.selected)
    project.setFilename(project_path)
    project.save()
    msg.value = f"Project saved to {project_path}"
project_choose.register_callback(save_project)
display(msg)
display(project_choose)

## Step 3: Setup a SO-CPA Attack and Execute It
To begin, run the first code block to display a dropdown menu for selecting the SO-CPA algorithm implementation, such as OpenMP, CUDA, etc. Refer to the documentation for detailed descriptions of each implementation.

Before executing the second code block, ensure you have selected the desired implementation from the dropdown menu.
Although you can choose smaller size of window size, too small window size may result in a failure of the attack.

Next, run the second code block to execute the SO-CPA attack.
We use the simple hamming weight model of sbox output for the attack.

Unlike the CPA tutorial, this implementation of the SO-CPA attack does not support progressive calculation of the correlation coefficients. Therefore, the attack results will only be displayed after the attack is completed.

The last print statement will display the combined sample points which yield the maximum correlation coefficient.

In [ ]:
from cw_plugins.analyzer.attacks.socpa import *
from ipywidgets import Dropdown, HBox, Label
algorithm_select = Dropdown(
    options=[
        ("SOCPAAlogrithm", SOCPAAlogrithm),
        ("SOCPAAlogrithmCuda", SOCPAAlogrithmCuda),
        ("SOCPAAlogrithmCudaFP32", SOCPAAlogrithmCudaFP32),
        ("SOCPAAlogrithmCudaNoSM", SOCPAAlogrithmCudaNoSM),
        ("SOCPAAlogrithmCudaFP32NoSM", SOCPAAlogrithmCudaFP32NoSM),
        ("SOCPAAlogrithmOpenCL", SOCPAAlogrithmOpenCL),
        ("SOCPAAlogrithmOpenCLFP32", SOCPAAlogrithmOpenCLFP32),
        ("SOCPAAlogrithmOpenCLNoSM", SOCPAAlogrithmOpenCLNoSM),
        ("SOCPAAlogrithmOpenCLFP32NoSM", SOCPAAlogrithmOpenCLFP32NoSM)
    ],
    value=SOCPAAlogrithm,  # Default value
)
display(HBox([Label("SO-CPA Algorithm"), algorithm_select]))

In [ ]:
from cw_plugins.analyzer.attacks.socpa import SOCPA
import chipwhisperer.analyzer as cwa
import time

# select the leakage model
leakage_model = cwa.leakage_models.sbox_output

# create an attack object
attack = SOCPA(project, leakage_model, algorithm_select.value)

# set window size
attack.window_size = 300

cb = cwa.get_jupyter_callback(attack)

# run the attack
start = time.time()
results = attack.run(callback=cb)
end = time.time()
print(f"Attack completed in {end - start:.2f} seconds.")

print(results)